In [ ]:
#uses MatrixVectorMultiply_#M_#N_#prPR_S#noepsE_HRx#hrf_C#mamcs_A#baseAdderSize(_NVL, VL) - v2.5

In [ ]:
import sys
import math
import os

In [ ]:
import import_ipynb
from MatrixVectorMultiply_SoE_V2_5 import MatrixVectorMultiply_SoE

In [ ]:
def MatrixMatrixMultiply_SoE(parameters, Print_To_File, Files_Location, Is_Top, modules):

    M = parameters["M"]
    N = parameters["N"]
    Q = parameters["Q"]
    PR = parameters["PR"]
    PC = parameters["PC"]
    NMult = parameters["NOEPS"]
    MAMCS = parameters["MAMCS"]
    AdderSize = parameters["ADDER_SIZE"]
    Enable_Column_Latch = parameters["ENABLE_COLUMN_LATCH"]
    Enable_Row_Latch = parameters["ENABLE_ROW_LATCH"]

    #IN_WIDTH>=1
    IN_WIDTH = 10
    #INPUT_REG_DEPTH>=0
    INPUT_REG_DEPTH = 1
    #MULT_PIPE_DEPTH>=0
    MULT_PIPE_DEPTH = 1
    
    HRR = math.ceil(N/NMult);
    RS = math.ceil(M/PR)
    CS = math.ceil(Q/PC)
    #NumOfComponents = math.ceil(NMult/MAMCS)

    ModuleName="MatrixMatrixMultiply_{}_{}_{}_{}PR_{}PC_S{}E_HRx{}_C{}_A{}_".format(M, N, Q, PR, PC, NMult, HRR, MAMCS, AdderSize)
    if Enable_Column_Latch<=0:
        ModuleName+="N"
    ModuleName+="CL_"
    if Enable_Row_Latch<=0:
        ModuleName+="N"        
    ModuleName+="RL"

    if Is_Top:
        Files_Location += ModuleName+"/"
        if not os.path.exists(Files_Location):
            os.makedirs(Files_Location)

    if Print_To_File<=0:
        of = sys.stdout
    else:
        of = open(Files_Location+ModuleName+".v", 'w+')

    print("`timescale 1ns / 1ps\n", file=of)
    print("module "+ModuleName, file=of)
    print("#(", file=of)
    print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
    print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
    print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
    print(")(", file=of)
    print("input clk, reset, enable,", file=of)
    print("output newInSeriesStart,", file=of)
    lghrr=math.ceil(math.log2(HRR))
    if lghrr==1:
        print("output inSeries,", file=of)
    elif lghrr>1:
        print("output [{}:0] inSeries,".format(lghrr-1), file=of)
    lgRS=math.ceil(math.log2(RS))
    if lgRS<=1:
        print("output rowSetInNo,", file=of)
    else:
        print("output [{}:0] rowSetInNo,".format(lgRS-1), file=of)
    lgCS=math.ceil(math.log2(CS))
    if lgCS<=1:
        print("output reg columnSetInNo = 0,", file=of)
    else:
        print("output reg [{}:0] columnSetInNo = 0,".format(lgCS-1), file=of)
    print("input inReady,", file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(PR):
        for j in range(NMult):
            print("AE{}R{}, ".format(j, i), end='', file=of)
        print(file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(PC):
        for j in range(NMult):
            print("BE{}C{}, ".format(j, i), end='', file=of)
        print(file=of)
    ac=CS*PC-Q
    QE = PC-ac-1
    if QE==0:
        stc1="C0"
    else:
        stc1="C0toC{}".format(QE)
    if QE==(PC-2):
        stc2="C{}".format(PC-1)
    else:
        stc2="C{}toC{}".format(QE+1, PC-1)
    ar=RS*PR-M
    ME = PR-ar-1
    if ME==0:
        str1="R0"
    else:
        str1="R0toR{}".format(ME)
    if ME==(PR-2):
        str2="R{}".format(PR-1)
    else:
        str2="R{}toR{}".format(ME+1, PR-1)
    print("output ", end='', file=of)
    print(stc1, end='', file=of)
    print(str1, end='', file=of)
    print("EReady,", file=of)
    if ar!=0:
        print("output ", end='', file=of)
        print(stc1, end='', file=of)
        print(str2, end='', file=of)
        print("EReady,", file=of)
    else:
        print("output reg NUEReady1 = 0, //not used", file=of)
    if ac!=0:
        print("output ", end='', file=of)
        print(stc2, end='', file=of)
        print(str1, end='', file=of)
        print("EReady,", file=of)
        if ar!=0:
            print("output ", end='', file=of)
            print(stc2, end='', file=of)
            print(str2, end='', file=of)
            print("EReady,", file=of)
        else:
            print("output reg NUEReady2 = 0, //not used", file=of)
    else:
        print("output reg NUEReady2 = 0, //not used", file=of)
        print("output reg NUEReady3 = 0, //not used", file=of)
    if lgRS<=1:
        print("output rowSetOutNo,", file=of)
    else:
        print("output [{}:0] rowSetOutNo,".format(lgRS-1), file=of)
    if lgCS<=1:
        print("output reg columnSetOutNo = {},".format(CS-1), file=of)
    else:
        print("output reg [{}:0] columnSetOutNo = {},".format(lgCS-1, CS-1), file=of)
    lgN=math.ceil(math.log2(N))
    al=-1+lgN
    if al==0:
        print("output signed [(2*IN_WIDTH):0] ", end='',file=of)
    elif al<0:
        print("output signed [(2*IN_WIDTH){}:0] ".format(al), end='',file=of)
    else:
        print("output signed [(2*IN_WIDTH)+{}:0] ".format(al), end='',file=of)
    for i in range(PC):
        for j in range(PR):
            print("EC{}R{}, ".format(i, j), end='',file=of)
    print(file=of)
    print("output early", end='', file=of)
    print(stc1, end='', file=of)
    print(str1, end='', file=of)
    print("EReady,", file=of)
    if ar!=0:
        print("output early", end='', file=of)
        print(stc1, end='', file=of)
        print(str2, end='', file=of)
        print("EReady,", file=of)
    else:
        print("output reg NUeEReady1 = 0, //not used", file=of)
    if ac!=0:
        print("output early", end='', file=of)
        print(stc2, end='', file=of)
        print(str1, end='', file=of)
        print("EReady,", file=of)
        if ar!=0:
            print("output early", end='', file=of)
            print(stc2, end='', file=of)
            print(str2, end='', file=of)
            print("EReady", file=of)
        else:
            print("output reg NUeEReady2 = 0 //not used", file=of)
    else:
        print("output reg NUeEReady2 = 0, //not used", file=of)
        print("output reg NUeEReady3 = 0 //not used", file=of)
    print(");\n", file=of)

    if Enable_Row_Latch>0:
        for j in range(PR):
            print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
            for i in range(NMult-1):
                print("SAE{}R{}, ".format(i, j), end='', file=of)
            print("SAE{}R{};".format(NMult-1, j), file=of)
        print(file=of)

    print("wire newInSeriesStartI;", file=of)
    if Enable_Row_Latch<=0 or Enable_Column_Latch<=0:
        print("wire inReadyC = inReady;", file=of)
        print("assign newInSeriesStart = newInSeriesStartI;", file=of)
    else: #Enable_Row_Latch>0 and Enable_Column_Latch>0
        #inrCC: inReady Check Cancel!
        print("wire inrCC = (columnSetInNo!=0) & (rowSetInNo!=0);", file=of) # ~inrCC : (columnSetInNo==0) | (rowSetInNo==0)
        print("wire inReadyC = inReady | inrCC;", file=of)
        print("assign newInSeriesStart = newInSeriesStartI & (~inrCC);", file=of)

    if(ac!=0): #CS*PC > Q, CS>=2
        print("wire inReadyC2 = (columnSetInNo!={}) & inReadyC;".format(CS-1), file=of)
        print(file=of)

    for i in range (PC):
        if Enable_Column_Latch>0:
            print("MatrixVectorMultiply_{}_{}_{}PR_S{}E_HRx{}_C{}_A{}_VL#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )".format(M, N, PR, NMult, HRR, MAMCS, AdderSize), file=of)
        else:
            print("MatrixVectorMultiply_{}_{}_{}PR_S{}E_HRx{}_C{}_A{}_NVL#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )".format(M, N, PR, NMult, HRR, MAMCS, AdderSize), file=of)
        print("MVM_{}(clk, reset, enable,".format(i), file=of)
        if i==0:
            print("newInSeriesStartI,", file=of)
            print("inSeries,".format(i), file=of)
            print("rowSetInNo,", file=of)
        else:
            print("newInSeriesStart{}, //not used".format(i), file=of)
            print("inSeries{}, //not used".format(i), file=of)
            print("rowSetInNo{}, //not used".format(i), file=of)
        if(PC-i<=ac):
            print("inReadyC2,", file=of)
        else:
            print("inReadyC,", file=of)
        for j in range(PR):
            if Enable_Row_Latch>0:
                for k in range(NMult):
                    print("SAE{}R{}, ".format(k, j), end='', file=of)
            else:
                for k in range(NMult):
                    print("AE{}R{}, ".format(k, j), end='', file=of)
            print(file=of)
        for j in range(NMult):
            print("BE{}C{}, ".format(j, i), end='', file=of)
        print(file=of)
        if i==0:
            print(stc1+str1+"EReady,", file=of)
            if ar!=0:
                print(stc1+str2+"EReady,", file=of)
            else:
                print("LLMVMoutReadyA{}, //not used".format(i), file=of)
        elif i==PC-ac:
            print(stc2+str1+"EReady,", file=of)
            if ar!=0:
                print(stc2+str2+"EReady,", file=of)
            else:
                print("LLMVMoutReadyCA{}, //not used".format(i), file=of)
        else:
            print("LLMVMoutReady{}, //not used".format(i), file=of)
            print("LLMVMoutReadyA{}, //not used".format(i), file=of)
        if i==0:
            print("rowSetOutNo,", file=of)
        else:
            print("LLrowSetOutNo{}, //not used".format(i), file=of)
        for j in range(PR):
            print("EC{}R{},".format(i, j), file=of)
        if i==0:
            print("early"+stc1+str1+"EReady,", file=of)
            if ar!=0:
                print("early"+stc1+str2+"EReady", file=of)
            else:
                print("LLMVMearlyOutReadyA{} //not used".format(i), file=of)
        elif i==PC-ac:
            print("early"+stc2+str1+"EReady,", file=of)
            if ar!=0:
                print("early"+stc2+str2+"EReady", file=of)
            else:
                print("LLMVMearlyOutReadyCA{} //not used".format(i), file=of)
        else:
            print("LLMVMearlyOutReady{},".format(i), file=of)
            print("LLMVMearlyOutReadyA{} //not used".format(i), file=of)
        print(");", file=of)
        print(file=of)

    #     if(RS==1):
    #         print("inReady & (columnSetInNo=={})".format(CS-1), end='', file=of)
    #     else:
    #         print("(rowSetInNo=={}) & inReady & (columnSetInNo=={})".format(RS-1, CS-1), end='', file=of)

    #     if(RS==1):
    #         print("inReady & (columnSetInNo!={})".format(CS-1), end='', file=of)
    #     else:
    #         if Enable_Row_Latch<=0 or Enable_Column_Latch<=0:
    #             print("(rowSetInNo=={}) & inReady & (columnSetInNo!={})".format(RS-1, CS-1), end='', file=of)
    #         else:
    #             print("(rowSetInNo=={})".format(RS-1), end='', file=of)

    # OLD:
    #     print("\t\tif(inReady", end='', file=of)
    #     if(RS>1):
    #         print(" & (rowSetInNo=={})".format(RS-1), end='', file=of)
    #     print(") begin", file=of)

    if(CS>1):
        if Enable_Row_Latch>0:
            for j in range(M):
                print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
                for i in range(N-1):
                    print("LAE{}R{}, ".format(i, j), end='', file=of)
                print("LAE{}R{};".format(N-1, j), file=of)
            print(file=of)
        print("always @(posedge clk) begin", file=of)
        print("\tif(reset) begin", file=of)
        print("\t\tcolumnSetInNo <= 0;", file=of)
        print("\tend", file=of)
        print("\telse if(enable) begin", file=of)

        print("\t\tif(", end='', file=of)
        if(RS==1):
            print("inReady & (inSeries=={})".format(HRR-1), end='', file=of)
        else:
            if Enable_Row_Latch<=0 or Enable_Column_Latch<=0:
                print("(rowSetInNo=={}) & inReady & (inSeries=={})".format(RS-1, HRR-1), end='', file=of)
            else:
                print("(rowSetInNo=={}) & (inReady | (columnSetInNo!=0)) & (inSeries=={})".format(RS-1, HRR-1), end='', file=of)
        print(") begin", file=of)

        print("\t\t\tif(columnSetInNo=={}) begin".format(CS-1), file=of)
        print("\t\t\t\tcolumnSetInNo <= 0;", file=of)
        print("\t\t\tend", file=of)
        print("\t\t\telse begin", file=of)
        if CS==2:
            print("\t\t\t\tcolumnSetInNo <= 1;", file=of)
        else:
            print("\t\t\t\tcolumnSetInNo <= columnSetInNo+1;", file=of)
        print("\t\t\tend", file=of)
        print("\t\tend", file=of)
        if Enable_Row_Latch>0:
            print("\t\tif(inReady & (columnSetInNo==0)) begin", file=of)
            if RS>1:
                print("\t\t\tcase(rowSetInNo)", file=of)
            for k in range(RS):
                if RS>1:
                    print("\t\t\t{}: begin".format(k), file=of)

                if RS>1:
                    print("\t", end='', file=of)
                print("\t\t\tcase(inSeries)", file=of)
                for h in range(HRR):
                    if RS>1:
                        print("\t", end='', file=of)
                    print("\t\t\t{}: begin".format(h), file=of)
                    t=k*PR-1
                    for j in range(PR):
                        t=t+1
                        if RS>1:
                            print("\t", end='', file=of)
                        print("\t\t\t\t", end='', file=of)
                        p=NMult*h-1
                        for l in range(NMult):
                            p=p+1
                            print("LAE{}R{} <= AE{}R{}; ".format(p,t,l,j), end='', file=of)
                            if p == N-1:
                                break
                        print(file=of)
                        if t==M-1:
                            break
                    if RS>1:
                        print("\t", end='', file=of)
                    print("\t\t\tend", file=of)
                if RS>1:
                    print("\t", end='', file=of)
                print("\t\t\tendcase", file=of)

                if RS>1:
                    print("\t\t\tend", file=of)
            if RS>1:
                print("\t\t\tendcase", file=of)
            print("\t\tend", file=of)
        print("\tend", file=of)
        print("end\n", file=of)

    if Enable_Row_Latch>0:
        print("always @(*) begin", file=of)
        print("\tif(columnSetInNo==0) begin", file=of)

        for j in range(PR):
            print("\t\t", end='', file=of)
            for i in range(NMult):
                print("SAE{}R{} = AE{}R{}; ".format(i, j, i, j), end='', file=of)
            print(file=of)

        print("\tend", file=of)
        print("\telse begin", file=of)

        if RS>1:
            print("\t\tcase(rowSetInNo)", file=of)

        for k in range(RS):
            if RS>1:
                print("\t\t{}: begin".format(k), file=of)
            if RS>1:
                print("\t", end='', file=of)
            print("\t\tcase(inSeries)", file=of)
            for h in range(HRR):
                if RS>1:
                    print("\t", end='', file=of)
                print("\t\t{}: begin".format(h), file=of)
                t=k*PR-1
                for j in range(PR):
                    t=t+1
                    if RS>1:
                        print("\t", end='', file=of)
                    print("\t\t\t", end='', file=of)
                    p=NMult*h-1
                    for l in range(NMult):
                        p=p+1
                        if p<N:
                            print("SAE{}R{} = LAE{}R{}; ".format(l,j,p,t), end='', file=of)
                        else:
                            print("SAE{}R{} = 0; ".format(l,j), end='', file=of)
                    print(file=of)
                    if t==M-1:
                        break

                if RS>1:
                    print("\t", end='', file=of)
                print("\t\tend", file=of)

            if (HRR & (HRR-1)) != 0: #check if HRR is not power of 2
                if RS>1:
                    print("\t", end='', file=of)
                print("\t\tdefault: begin".format(k), file=of)
                for jp in range(j+1):
                    if RS>1:
                        print("\t", end='', file=of)
                    print("\t\t\t", end='', file=of)
                    for l in range(NMult):
                        print("SAE{}R{} = 0; ".format(l, jp), end='', file=of)
                    print(file=of)
                if RS>1:
                    print("\t", end='', file=of)
                print("\t\tend", file=of)

            if RS>1:
                print("\t", end='', file=of)
            print("\t\tendcase", file=of)

            for jj in range(j+1, PR): #last iteration, M!=RS*PR
                if RS>1:
                    print("\t", end='', file=of)
                print("\t\t", end='', file=of)
                for l in range(NMult):
                    print("SAE{}R{} = 0; ".format(l,jj), end='', file=of)
                print(file=of)

            if RS>1:
                print("\t\tend", file=of)

        if RS>1:
            if (RS & (RS-1)) != 0: #check if RS is not power of 2
                print("\t\tdefault: begin".format(k), file=of)
                for j in range(PR):
                    print("\t\t\t", end='', file=of)
                    for i in range(NMult):
                        print("SAE{}R{} = 0; ".format(i, j), end='', file=of)
                    print(file=of)
                print("\t\tend", file=of)

            print("\t\tendcase", file=of)
        print("\tend", file=of)
        print("end\n", file=of)

    if(CS>1):
        print("always @(posedge clk) begin", file=of)
        print("\tif(reset) begin", file=of)
        print("\t\tcolumnSetOutNo <= {};".format(CS-1), file=of)
        print("\tend", file=of)
        print("\telse if(enable) begin", file=of)
        print("\t\tif(early"+stc1+str1+"EReady & (rowSetOutNo=={})) begin".format(RS-1), file=of)
        print("\t\t\tif(columnSetOutNo=={}) begin".format(CS-1), file=of)
        print("\t\t\t\tcolumnSetOutNo <= 0;", file=of)
        print("\t\t\tend", file=of)
        print("\t\t\telse begin", file=of)
        if CS==2:
            print("\t\t\t\tcolumnSetOutNo <= 1;", file=of)
        else:
            print("\t\t\t\tcolumnSetOutNo <= columnSetOutNo+1;", file=of)
        print("\t\t\tend", file=of)
        print("\t\tend", file=of)
        print("\tend", file=of)
        print("end\n", file=of)

    print("endmodule", file=of)

    if Print_To_File>0:
        of.close()
        
    MatrixVectorMultiply_SoE(parameters, Print_To_File, Files_Location, 0, modules)
    
    modules[ModuleName]="MatrixMatrixMultiply_SoE"
    return ModuleName